In [ ]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving movies.csv to movies.csv


In [ ]:
uploaded1 = files.upload()

Saving ratings.csv to ratings.csv


In [ ]:
df_movie = pd.read_csv('movies.csv', usecols=['movieId','title'])
df_movie.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [ ]:
titles = df_movie['title'][10]
titles

'American President, The (1995)'

In [ ]:
df_movie.shape

(9742, 2)

In [ ]:
df_rating = pd.read_csv('ratings.csv')
df_rating = df_rating.drop(['timestamp'], axis=1)
print(df_rating.shape)
df_rating.head()

(100836, 3)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
movies_users=df_rating.pivot(index='movieId', columns='userId',values='rating').fillna(0)
movies_users.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.5,3.5,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.5,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,2.5,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
m_movies_users=csr_matrix(movies_users.values)

In [ ]:
m_movies_users

<9724x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [ ]:
model_knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

In [ ]:
model_knn.fit(m_movies_users)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                 radius=1.0)

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import process

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
def recommend(movie_name, recommendations ):
    #model_knn.fit(data)
    id=process.extractOne(movie_name, df_movie['title'])[2]
    print('Movie Selected: ',df_movie['title'][id])#, 'Index: ',id)
    print('Searching for recommendations.....')
    model_knn.fit(m_movies_users)
    distances, indices=model_knn.kneighbors(m_movies_users[id], n_neighbors = recommendations)
    for i in indices:
        print(df_movie['title'][i].where(i!=id))

In [ ]:
recommend('iron man', 20)

Movie Selected:  Iron Man (2008)
Searching for recommendations.....
6743                                            NaN
7197                                  Garage (2007)
7195                        Merry Madagascar (2009)
7354                             A-Team, The (2010)
6726                         Superhero Movie (2008)
7137                         Thirst (Bakjwi) (2009)
7026                                 Scorpio (1973)
7571                                 Win Win (2011)
3880                  Look Who's Talking Now (1993)
6388    After the Wedding (Efter brylluppet) (2006)
7601                       Idiots and Angels (2008)
6755                Nina's Heavenly Delights (2006)
7022                                   Earth (2007)
7338                          Blue Valentine (2010)
4421                         What's Up, Doc? (1972)
6195                             Silent Hill (2006)
5885                                   Crash (2004)
3740                              Motel Hell (19

In [ ]:
recommend('ironman', 10)

Movie Selected:  Tetsuo, the Ironman (Tetsuo) (1988)
Searching for recommendations.....
3354                                                  NaN
5170                           Garfield: The Movie (2004)
5542                                    Moonwalker (1988)
4340         Man on the Train (Homme du train, L') (2002)
3805                     Life or Something Like It (2002)
3308                                  Big Business (1988)
4756                            That Touch of Mink (1962)
5040                       Tremors II: Aftershocks (1996)
5233    Happenstance (Battement d'ailes du papillon, L...
1535                                Tender Mercies (1983)
Name: title, dtype: object


In [ ]:

idx=process.extractOne('iron man', df_movie['title'])[2]
print('Movie Selected: ',df_movie['title'][idx])#, 'Index: ',id)
print('Searching for recommendations.....')
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors = 20)
model_knn.fit(m_movies_users)
distances, indices=model_knn.kneighbors(m_movies_users[idx], n_neighbors = 5)
titls = []
for i in indices:
    titls.append(df_movie['title'][i].where(i!=idx))
    print(df_movie['title'][i].where(i!=idx))
movie_title = pd.DataFrame(titls, columns=['title'])
m = movie_title['title']
print(m)

Movie Selected:  Iron Man (2008)
Searching for recommendations.....
6743                        NaN
7197              Garage (2007)
7195    Merry Madagascar (2009)
7354         A-Team, The (2010)
6726     Superhero Movie (2008)
Name: title, dtype: object
title   NaN
Name: title, dtype: float64


In [ ]:
for i in indices:
    titls.append(df_movie['title'][i])
    print(df_movie['title'][i])
movie_title = pd.DataFrame(titls, columns=['title'])
m = movie_title['title']
print(m)

6743            Iron Man (2008)
7197              Garage (2007)
7195    Merry Madagascar (2009)
7354         A-Team, The (2010)
6726     Superhero Movie (2008)
Name: title, dtype: object
title   NaN
title   NaN
title   NaN
Name: title, dtype: float64


In [ ]:
type(indices)

numpy.ndarray

In [ ]:
indices

array([[6743, 7197, 7195, 7354, 6726]])

In [ ]:
df_movie['title'][indices[0]]

6743            Iron Man (2008)
7197              Garage (2007)
7195    Merry Madagascar (2009)
7354         A-Team, The (2010)
6726     Superhero Movie (2008)
Name: title, dtype: object

In [ ]:
len(indices)

1

In [ ]:
distances

array([[7.77156117e-16, 4.09144753e-01, 4.13091640e-01, 4.18127003e-01,
        4.35628075e-01]])

In [ ]:
ind = indices.tolist()
ind

[[6743, 7197, 7195, 7354, 6726]]

In [ ]:
ind[][0]

IndexError: ignored

In [ ]:
li =[]
for i in ind[0]:
    li.append(df_movie['title'][i])
    #print(df_movie['title'][i])
del li[0]
print(li)

['Garage (2007)', 'Merry Madagascar (2009)', 'A-Team, The (2010)', 'Superhero Movie (2008)']


In [ ]:
li =[]
for i in indices[0]:
    li.append(df_movie['title'][i])
    #print(df_movie['title'][i])
del li[0]
print(li)

['Garage (2007)', 'Merry Madagascar (2009)', 'A-Team, The (2010)', 'Superhero Movie (2008)']
